# VOLTANDO UMA PASTA PARA TRAZ

In [ ]:
import os
os.chdir("../")

# PIPELINE EXTRAÇÃO - DOTA 2

In [ ]:
from modules.downloader import FileDownloader
from modules.extractor import ZipExtractor
import shutil
import json

path_download   = os.path.join("data","external")
link_download   = "https://github.com/odota/dotaconstants/archive/refs/heads/master.zip"

FD              = FileDownloader()
status, path    = FD.download(link_download, path_download)
if status:
    # Define the path to the downloaded zip file
    path_extract= os.path.join(os.path.dirname(path), "dotaconstants")
    EX          = ZipExtractor(path, path_extract)
    dir_extract = EX.extract()
    # Check if the extraction directory contains files
    if len(os.listdir(dir_extract)) > 0:
        # Define the path to the extracted files and the destination directory
        path_extract_build  = os.path.join(dir_extract, os.listdir(dir_extract)[0], "build")
        path_to             = os.path.join("constants","games","dota2")
        # If the extraction path exists, copy the files to the destination directory
        if os.path.exists(path_extract_build):
            os.makedirs(path_to, exist_ok=True)
            # Copy files from the extraction path to the destination directory
            for item in os.listdir(path_extract_build):
                src = os.path.join(path_extract_build, item)
                dst = os.path.join(path_to, item)
                if os.path.isfile(src):
                    shutil.copy2(src, dst)
            # Verificar se todos os arquivos foram copiados com sucesso
            arquivos_copiados   = set(os.listdir(path_to))
            arquivos_esperados  = set(os.listdir(path_extract_build))
            if arquivos_copiados == arquivos_esperados:
                # Excluir a pasta path_extract e o arquivo path
                shutil.rmtree(path_extract)
                os.remove(path)
            else:
                print("Alguns arquivos não foram copiados:", arquivos_esperados - arquivos_copiados)

# INSERINDO NO BANCO DE DADOS

VERSÕES

In [ ]:
from modules import db_manager
from jinja2 import Template
from tqdm import tqdm

versions                = {}
path_fille              = os.path.join("constants", "games", "dota2", "patch.json")
path_insert             = "repository\\querys\\operation\\insert\\game_versions.sql"

# Read the SQL query template from the file
with open(path_insert, "r", encoding="utf-8") as file:
    query_insert        = file.read()

# Read the JSON file containing game versions
if os.path.exists(path_fille):
    with open(path_fille, "r", encoding="utf-8") as file:
        versions        = json.load(file)

if versions:
    for version in tqdm(versions, desc="Inserting Game Versions"):
        sql_template    = Template(query_insert)
        rendered_sql    = sql_template.render(
            ID          = version["id"]                                     if "id"             in version else None,
            NAME        = version["name"].replace("'", "").strip()          if "name"           in version else None,
            DATE        = version["date"].replace("'", "").strip()          if "date"           in version else None,
            ID_GAME     = 1,
        ).replace("'None'", "NULL").replace("None", "NULL")\
        .replace("'False'", "FALSE").replace("False", "FALSE")\
        .replace("'True'", "TRUE").replace("True", "TRUE")
        
        try:
            db_manager.execute_raw(rendered_sql)
        except Exception as e:
            print(f"Error inserting video data: {e}")
            db_manager.connection.rollback()

# BAIXANDO IMAGE DOS ITENS

In [ ]:
repository = ["https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/items/{}.png"]

# BAIXANDO IMAGE DOS HEROIS

In [ ]:
repository = ["https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/{}.png"]


# BAIXANDO IMAGE DAS ABILIDADES

In [ ]:
repository = ["https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/abilities/{}.png"]

# BAIXANDO IMAGE DOS TEAMS

In [ ]:
repository = [
    "https://steamcdn-a.akamaihd.net/apps/dota2/images/team_logos/{}.png",
    "https://cdn.steamusercontent.com/ugc/{}/{}/" # 2909225722380320  5C6EFC9004093ED29E9B48242DE79418BDDBFE30
]